For this homework, make sure that you format your notbook nicely and cite all sources in the appropriate sections. Programmatically generate or embed any figures or graphs that you need.

Names: __Lucas Calero Forero, Rebecca McBrayer__

Step 1: Word2Vec paper questions
---------------------------
1. Describe how a CBOW word embedding is generated.

A feedforward Neural Net without a hidden layer with a shared projection layer is used to classify the current word on every iteration based on its context (previous and future words). The output is a word embedding of size V.

2. What is a CBOW word embedding and how is it different from a skip-gram word embedding?

The biggest difference between skip-gram and CBOW is the fact that skip-gram uses the current word as the input into the classifier and predicts words within a range R of the word 1 < R < C where C is the maxiumum distance. This technique allows for weighted classification depending on the distance from w(i) and the current word.  

3. What is the task that the authors use to evaluate the generated word embeddings?

The authors use the Semantic-Syntactic Word Relationship test set to evaluate their generated word embeddings, a test set developed by them for this paper and for future research.

4. What are PCA and t-SNE? Why are these important to the task of training and interpreting word embeddings?

PCA and t-SNE are methods of dimensionality reduction, projecting high-dimensional word embeddings into lower dimensional space. This is important foor training and interpreting word embeddings since embeddings can become extremely high dimensional, and projecting them into a lower space will both speed up training time and make it easier for humans to visualize and therefore interpret.

Step 2: Train your own word embeddings
--------------------------------

The Spooky Authors dataset is an old competition dataset from Kaggle. It has example sentences from famous works of writing from three different authors. For this assignment, each data point is a single sentence written by one of the three gothic authors.

Describe what data set you have chosen to compare and contrast with the Spooky Authors Dataset. Make sure to describe where it comes from and its general properties.

We chose the text of Les Miserables as our second dataset. This comes from Project Gutenburg and has over 500,000 words in its translated form.

In [140]:
# import your libraries here
import string
import urllib.request

from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize
import numpy as np
import pandas as pd
import tensorflow as tf

# Importing utility functions from Keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Embedding


### a) Train embeddings on GIVEN dataset

In [53]:
# Read the file 'spooky-author-identification/train.csv' 
spooky = pd.read_csv('train.csv')

In [54]:
# code to train your word embeddings
spooky_text = list(spooky['text'])

# Remove punctuation
exclude = set(string.punctuation)
spooky_nopun = []
for st in spooky_text:
  st = ''.join(ch for ch in st if ch not in exclude)
  spooky_nopun.append(st)

# Add start and ending tokens, and make all words lowercase
spooky_text = ["<s> {} </s>".format(sentence.lower()) for sentence in spooky_nopun]
spooky_data = [sentence.split() for sentence in spooky_text]

In [55]:
# The dimension of word embedding. 
# This variable will be used throughout the program
# you may vary this as you desire
EMBEDDINGS_SIZE = 200

# Train the Word2Vec model from Gensim. 
# Below are the hyperparameters that are most relevant. 
# But feel free to explore other 
# options too:
# sg = 1
# window = 5
# size = EMBEDDING_SIZE
# min_count = 1

model = Word2Vec(sentences=spooky_data, size=EMBEDDINGS_SIZE, window=5, min_count=1, workers=4)
model.save("word2vec.model")

/Users/lucascaleroforero/opt/anaconda3/envs/threefive/lib/python3.5/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [56]:
# if you save your Word2Vec as the variable model, this will 
# print out the vocabulary size
print('Vocab size {}'.format(len(model.wv.vocab)))

Vocab size 25413


In [57]:
# You can save file in txt format, then load later if you wish.
model.wv.save_word2vec_format('embeddings.txt', binary=False)

/Users/lucascaleroforero/opt/anaconda3/envs/threefive/lib/python3.5/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### b) Train embedding on YOUR dataset

In [58]:
# Pull the data from Project Gutenburg
with urllib.request.urlopen("http://www.gutenberg.org/files/135/135-0.txt") as f:
  content = f.read()

text = content.decode('utf-8-sig')

# Remove parts of the text that aren't the book itself, plus extraneous whitespace
text = text[text.find("So long as there shall exist"):text.find("Publisher of the Italian translation ")]
text = ''.join(ch for ch in text if ch not in ['\n','\r'])

# Tokenize
sentences = sent_tokenize(text)

In [59]:
# Remove punctuation
exclude = set(string.punctuation)
text_nopun = []
for st in sentences:
  st = ''.join(ch for ch in st if ch not in exclude)
  text_nopun.append(st)

# Add start and ending tokens, and make all words lowercase
text = ["<s> {} </s>".format(sentence.lower()) for sentence in text_nopun]
lesmis_data = [sentence.split() for sentence in text]
lesmis_data = list(filter(lambda a: len(a)>2, lesmis_data))  # Remove blank sentences

In [60]:
EMBEDDINGS_SIZE = 200

# Train the Word2Vec model from Gensim. 
# Below are the hyperparameters that are most relevant. 
# But feel free to explore other 
# options too:
# sg = 1
# window = 5
# size = EMBEDDING_SIZE
# min_count = 1

lesmis_model = Word2Vec(sentences=lesmis_data, size=EMBEDDINGS_SIZE, window=5, min_count=1, workers=4)
lesmis_model.save("lesmis_word2vec.model")

/Users/lucascaleroforero/opt/anaconda3/envs/threefive/lib/python3.5/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [61]:
lesmis_model.wv.save_word2vec_format('lesmis_embeddings.txt', binary=False)
print('Vocab size {}'.format(len(lesmis_model.wv.vocab)))

/Users/lucascaleroforero/opt/anaconda3/envs/threefive/lib/python3.5/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Vocab size 63355


What text-normalization and pre-processing did you do and why? __We chose to do only very minimal text preprocessing, to keep our implementation simple. We removed punctuation and made all words lowercase, since those have no impact on the meaning of words (and therefore how they should be represented by an embedding. We added start and end tokens to make learning easier, but otherwise left the sentences alone. We did not lemmatize, as the different forms of words will have different meanings and therefore we decided to let them be embedded differently.__

Step 3: Evaluate the differences between the word embeddings
----------------------------

(make sure to include graphs, figures, and paragraphs with full sentences)

Cite your sources:
-------------

Step 4: Feedforward Neural Language Model
--------------------------

### a) First, encode  your text into integers

In [62]:
NGRAM = 3 # The size of the ngram language model you want to train

flat_data = [word for sent in spooky_data for word in sent]
spooky_vocab = list(set(flat_data))

# Initializing a Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(flat_data)

indexed_spooky_data = tokenizer.texts_to_sequences(spooky_data)
indexed_spooky_vocab = tokenizer.texts_to_sequences(spooky_vocab)

word_to_index = {word:index[0] for word, index in zip(list(spooky_vocab), indexed_spooky_vocab)}
index_to_word = {index[0]:word for word, index in zip(list(spooky_vocab), indexed_spooky_vocab)}

### b) Next, prepare your sequences from text

#### Fixed ngram based sequences (Used for Feedforward)

The training samples will be structured in the following format. 
Depending on which ngram model we choose, there will be (n-1) tokens 
in the input sequence (X) and we will need to predict the nth token (Y)

            X,						  y
    this,    process               however
    process, however               afforded
    however, afforded	           me

In [63]:
def generate_ngram_training_samples(content: list) -> list:
    '''
    Takes the encoded data (list of lists) and 
    generates the training samples out of it.
    Parameters:
    up to you, we've put in what we used
    but you can add/remove as needed
    return: 
    list of lists in the format [[x1, x2, ... , x(n-1), y], ...]
    '''
    ngrams = []
    for sent in content:
        for i in range(NGRAM-1, len(sent)):
            ngram = [sent[j] for j in range(i-(NGRAM-1), i+1)]
            ngrams.append(ngram)
    return ngrams

### c) Then, split the sequences into X and y and create a Data Generator

In [64]:
def read_embeddings(path: string) -> (dict,dict):
    '''Loads and parses embeddings trained in earlier.
    Parameters and return values are up to you.
    '''
    # you may find generating the following two dicts useful:
    # word to embedding : {'the':1, ...}
    # index to embedding : {1:'the', ...} (inverse of word_2_embedding)
    
    model = Word2Vec.load(path)
    word_to_embedding = {}
    index_to_embedding = {}
    for word in model.wv.vocab.keys():
        index = word_to_index[word]
        embedding = model.wv[word]
        word_to_embedding[word] = embedding
        index_to_embedding[index] = embedding
    return word_to_embedding, index_to_embedding

### Create the x & y lists

In [65]:
spooky_ngrams = generate_ngram_training_samples(indexed_spooky_data)
encoded_X = []
encoded_Y = []
for ngram in spooky_ngrams:
    encoded_X.append(ngram[:NGRAM-1])
    encoded_Y.append(ngram[-1])

spooky_word_to_embedding, spooky_index_to_embedding = read_embeddings("word2vec.model")

/Users/lucascaleroforero/opt/anaconda3/envs/threefive/lib/python3.5/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [67]:
spooky_index_to_embedding[3]

array([-7.53004625e-02, -1.63967356e-01,  4.37836312e-02,  6.10692389e-02,
        2.14999080e-01, -7.32908189e-01, -2.33377665e-02,  2.38539085e-01,
        1.62604630e+00,  4.20291811e-01, -4.18647498e-01, -3.14808071e-01,
       -4.00383174e-01,  9.02660906e-01,  2.65009731e-01,  7.81773701e-02,
        4.04042959e-01,  4.11729902e-01, -6.47262752e-01,  7.25369632e-01,
       -1.28131006e-02,  1.48308015e+00, -9.96735245e-02,  3.83843631e-01,
        6.72053754e-01, -1.15318644e+00, -1.56387076e-01,  1.60512650e+00,
       -3.90817299e-02, -8.34428668e-01,  8.06908682e-02, -4.38925892e-01,
        3.46613944e-01,  5.19005299e-01,  5.37110269e-01, -7.84928977e-01,
       -5.14088199e-02, -1.09623504e+00, -6.66607141e-01, -1.16435006e-01,
        7.34403282e-02,  5.84439576e-01,  4.89185095e-01,  2.99685508e-01,
       -7.10928798e-01, -1.39066413e-01, -1.87709108e-01, -2.97799498e-01,
       -6.10906482e-01, -5.85647941e-01, -8.42804134e-01,  1.60257816e-01,
       -5.31381428e-01,  

In [130]:
def data_generator(X: list, Y: list, num_sequences_per_batch: int) -> (list,list):
    '''
    Returns data generator to be used by feed_forward
    https://wiki.python.org/moin/Generators
    https://realpython.com/introduction-to-python-generators/
    
    Yields batches of embeddings and labels to go with them.
    Use one hot vectors to encode the labels 
    (see the to_categorical function)
    
    '''
    num_sequences = 0
    
    while True:
        next_X = X[num_sequences: num_sequences+num_sequences_per_batch]
        next_X_vec = []
        for X_sent in next_X:
            X_sent_next = []
            for ii in X_sent:
                X_sent_next.extend(spooky_index_to_embedding[ii])
            next_X_vec.append(X_sent_next)
        
        next_Y = Y[num_sequences: num_sequences+num_sequences_per_batch]
#         next_Y_vec = [spooky_index_to_embedding[ii] for ii in next_Y]
        next_Y_cat = [tf.keras.utils.to_categorical(vec, num_classes=len(spooky_vocab)) for vec in next_Y]
        next_Y_tensor = [tf.keras.backend.constant(cat) for cat in next_Y_cat]
        
        yield tf.keras.backend.constant(next_X_vec),next_Y_tensor
        num_sequences += num_sequences_per_batch

In [131]:
# Examples
# initialize data_generator
sequences = indexed_spooky_data
num_sequences_per_batch = 128 # this is the batch size
steps_per_epoch = len(sequences)//num_sequences_per_batch  # Number of batches per epoch
train_generator = data_generator(encoded_X, encoded_Y, num_sequences_per_batch)

sample=next(train_generator) # this is how you get data out of generators
sample[0].shape # (batch_size, (n-1)*EMBEDDING_SIZE)

TensorShape([Dimension(128), Dimension(400)])

### d) Train your models

[[ 0.71050334 -0.7888891  -0.08812644 ...  0.01623069  0.0080067
  -0.11172828]
 [ 0.04235823 -0.06369358 -0.01139935 ...  0.02198141  0.03586606
  -0.7135335 ]
 [ 0.4342295  -1.0396879  -0.28747782 ...  0.0193022  -0.00820226
  -0.3251715 ]
 ...
 [ 0.06262499 -0.14558782 -0.02257299 ...  0.2982982   0.39763406
  -1.2082713 ]
 [ 0.6363854   0.4528262   0.81134176 ... -0.0103347   0.06096267
  -0.22863361]
 [ 0.082367   -0.08252271  0.01731132 ... -0.11803728 -0.10200374
  -0.91620755]]


In [132]:
# code to train a feedforward neural language model 
# on a set of given word embeddings
# make sure not to just copy + paste to train your two models

# Define the model architecture using Keras Sequential API

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, input_shape=(num_sequences_per_batch,(EMBEDDINGS_SIZE * (NGRAM-1)))))
model.add(tf.keras.layers.Activation('softmax'))
model.compile(optimizer='sgd', loss='binary_crossentropy')

In [133]:
# Start training the model
model.fit(x=train_generator,
          steps_per_epoch=steps_per_epoch,
          epochs=3)

AttributeError: 'generator' object has no attribute 'ndim'

### e) Generate Sentences

In [ ]:
# generate a sequence from the model
def generate_seq(model: Sequential, 
                 tokenizer: Tokenizer, 
                 seed: list, 
                 n_words: int):
    '''
    Parameters:
        model: your neural network
        tokenizer: the keras preprocessing tokenizer
        seed: [w1, w2, w(n-1)]
        n_words: generate a sentence of length n_words
    Returns: string sentence
    '''
    pass

### f) Compare your generated sentences

Sources Cited
----------------------------
